# 딥 러닝을 이용한 자연어 처리 입문

아래 링크의 E-book을 보고 실습한 내용입니다.

WikiDocs 주소: https://wikidocs.net/31766


# 5장 카운트 기반의 단어 표현

## 1절 코사인 유사도 (Cosine Similarity)

## 코사인 유사도 (Cosine Similarity)

### Numpy를 이용한 Cosine similarity 계산

계산결과에서 눈 여겨볼 점은 doc2와 doc3의 코사인 유사도를 구했을 때 1이 나온다는 점이다.
코사인 유사도는 벡터의 방향만을 고려하기 때문에 같은 벡터로 판단한 것이다.

이런 특성 덕분에 문서의 길이가 다를 때 문서의 길이가 다르다는 이유만으로 다른 문장으로
판단하지 않고, 좀 더 공정하게 유사도를 계산할 수 있다는 장점이 있다.

In [11]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    """코사인 유사도를 계산하는 함수"""
    return dot(A, B) / (norm(A) * norm(B))

In [12]:
doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

In [13]:
print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


## 코사인 유사도를 이용한 영화 추천 시스템 예제

- 케글 영화 데이터셋: https://www.kaggle.com/rounakbanik/the-movies-dataset

In [14]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [15]:
# pandas로 데이터를 불러오는 과정
data = pd.read_csv("movies_metadata.csv", low_memory=False)

# 데이터의 양을 2만개로 축소 시킨 후 실습
data = data.head(20000)
data.head(2)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [16]:
# TF-IDF를 계산할 때 null값이 들어있다면 에러가 발생하기 때문에 제거해줘야 한다.
print("Null값 수:", data['overview'].isnull().sum())
data['overview'] = data['overview'].fillna('')
print("Null값을 채운 후:", data['overview'].isnull().sum())


Null값 수: 135
Null값을 채운 후: 0


In [17]:
# overview 열에 대한 TF-IDF 계산 
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(data["overview"])
print("(영화수, 단어수) =", tfidf_matrix.shape)

(영화수, 단어수) = (20000, 47487)


In [18]:
# 코사인 유사도를 이용해 문서의 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [19]:
# 랜덤한 영화의 Title을 뽑아서 인덱스 구하기
random_movie = indices.sample(1)
print("영화의 제목:", random_movie.index[0])
print("영화의 인덱스 값:", random_movie[0])

영화의 제목: Valley Girl
영화의 인덱스 값: 6489


In [20]:
def get_recomendations(title, cosine_sim):
    """코사인 유사도를 기준으로 가장 비슷한 10개의 영화 추천"""
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[movie_indices]

get_recomendations(random_movie.index[0], cosine_sim)

9221       Possible Worlds
4760           Shallow Hal
16772               Liliom
13314      Punk's Not Dead
12323    God Save the King
16070              Tribute
10195        Julie Johnson
19800         Nature Calls
13153           The Lovers
7955       The Last Valley
Name: title, dtype: object